In [1]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

train_dir = 'C://Users//akash//OneDrive//Documents//archive//train'
val_dir = 'C://Users//akash//OneDrive//Documents//archive//test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}


emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

emotion_model.save('emotion_model.h5')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


C:\Users\akash\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
448/448 [==============================] - 504s 1s/step - loss: 1.8269 - accuracy: 0.2499 - val_loss: 1.7135 - val_accuracy: 0.3468
Epoch 2/50
448/448 [==============================] - 201s 448ms/step - loss: 1.6622 - accuracy: 0.3543 - val_loss: 1.5520 - val_accuracy: 0.4065
Epoch 3/50
448/448 [==============================] - 193s 430ms/step - loss: 1.5551 - accuracy: 0.4006 - val_loss: 1.4695 - val_accuracy: 0.4395
Epoch 4/50
448/448 [==============================] - 189s 422ms/step - loss: 1.4818 - accuracy: 0.4284 - val_loss: 1.4050 - val_accuracy: 0.4636
Epoch 5/50
448/448 [==============================] - 186s 416ms/step - loss: 1.4135 - accuracy: 0.4625 - val_loss: 1.3601 - val_accuracy: 0.4834
Epoch 6/50
448/448 [==============================] - 189s 422ms/step - loss: 1.3622 - accuracy: 0.4855 - val_loss: 1.3181 - val_accuracy: 0.5027
Epoch 7/50
448/448 [==============================] - 208s 465ms/step - loss: 1.3237 - accuracy: 0.4976 - val_loss: 1.2880 - va

In [2]:
import tensorflow as tf

emotion_model = tf.keras.models.load_model('emotion_model.h5')
emotion_model.layers[0].input_shape

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
emoji_dist={0:"C://Users//akash//OneDrive//Documents//archive//emojis//angry.png",1:"C://Users//akash//OneDrive//Documents//archive//emojis//disgusted.png",2:"C://Users//akash//OneDrive//Documents//archive//emojis//fearful.png",3:"C://Users//akash//OneDrive//Documents//archive//emojis//happy.png",4:"C://Users//akash//OneDrive//Documents//archive//emojis//neutral.png",5:"C://Users//akash//OneDrive//Documents//archive//emojis//sad.png",6:"C://Users//akash//OneDrive//Documents//archive//emojis//surpriced.png"}

In [ ]:
import PySimpleGUI as sg
import cv2
import numpy as np
from PIL import Image
import io

show_text=[0]  
cap = cv2.VideoCapture(0)

def main():

    sg.theme('systemdefault')

    # define the window layout
    layout = [[sg.Text('Emojify', size=(40, 1), justification='center', font='Helvetica 20')],
              [sg.Image(filename='', key='image'),sg.Image(filename='',key="emoji")],
              [sg.Button('Start', size=(10, 1), font='Helvetica 14'),
               sg.Button('Exit', size=(10, 1), font='Helvetica 14'), ]]

    # create the window and show it without the plot
    window = sg.Window('Real time emoji',
                       layout, location=(800, 400))

    # ---===--- Event LOOP Read and display frames, operate the GUI --- #
    recording = False

    while True:
        event, values = window.read(timeout=40)
        
        if event == 'Exit' or event == sg.WIN_CLOSED:
            break

        elif event == 'Start':
            recording = True
            
        if recording:
            flag1, frame1 = cap.read()
            frame1 = cv2.resize(frame1,(600,500))
            
            bounding_box = cv2.CascadeClassifier('C://Users//akash//OneDrive//Documents//archive//haarcascade_frontalface_default.xml')
            gray_frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
            
            for (x, y, w, h) in num_faces:
                cv2.rectangle(frame1, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
                roi_gray_frame = gray_frame[y:y + h, x:x + w]
                cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
                
                prediction = emotion_model.predict(cropped_img)
                maxindex = int(np.argmax(prediction))
                
                cv2.putText(frame1, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                show_text[0]=maxindex
    
                imgbytes = cv2.imencode('.png', frame1)[1].tobytes()
                window['image'].update(data=imgbytes)
                
                image = Image.open(emoji_dist[show_text[0]])
                image.thumbnail((400, 400))
                
                im = cv2.imread(emoji_dist[show_text[0]])
                cv2.imwrite("emo.png",im)
             
                bio = io.BytesIO()
                image.save(bio, format="PNG")
                window["emoji"].update(data=bio.getvalue())
                
                      
    cap.release()
    window.close()
    
main()